In [1]:
%reload_ext watermark
from metapool import KLSampleSheet, validate_and_scrub_sample_sheet
%watermark -i -v -iv -m -h -p metapool,sample_sheet,openpyxl -u

Last updated: 2021-12-15T19:48:14.923859-06:00

Python implementation: CPython
Python version       : 3.9.7
IPython version      : 7.30.1

metapool    : 0+untagged.112.g8fed443.dirty
sample_sheet: 0.12.0
openpyxl    : 3.0.9

Compiler    : Clang 10.0.0 
OS          : Darwin
Release     : 20.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 12
Architecture: 64bit

Hostname: Kelly-Fogelsons-MacBook-Pro.local



# Knight Lab Sample Sheet Validation

This notebook is designed to validate and troubleshoot sample sheets of externally generated plates.

The steps are as follows:

1. Parse sample sheet.
1. Check that all the required columns in the `Data` section are present.
1. Check that the `Bioinformatics` and `Contact` section are present.
1. Validate and scrub sample identifiers so they are compliant with Illumina's `bcl2fastq` software.
    - Automatically replace non-allowed characters for underscores.
    - Flag non-unique sample identifiers.
1. Check that lane values are not empty.
1. Check that projects in the `Data`, `Bioinformatics` and `Contact` sections are all valid.
1. Validate the Qiita study identifier suffix at the end of every project name.
1. Save the parsed file in a compliant format.

**Note**: warning and error messages (text highlighted in red) will inform you of any problems that may come up.

**Enter the correct path to the sample sheet you want to validate**, replace the path to `good-sample-sheet.csv` for the location of the sheet you want to validate.

In [2]:
sheet = KLSampleSheet('../metapool/tests/data/good-sample-sheet.csv')
valid_sheet = validate_and_scrub_sample_sheet(sheet)

If there are any error messages, please correct the sample sheet and re-run the cell above. Once you are happy with the results run the cell below, otherwise you will see an exception below.

In [3]:
with open('validated-sample-sheet.csv', 'w') as f:
    valid_sheet.write(f)